#**Классификация текста при помощи нейросетей**

#Название проекта:
**Разработка системы распознавания русскоязычного текста, сгенерированного LLM**

Область для которой будет реализован проект искусственного интеллекта и машинного обучения – образование.

**Актуальность** данной задачи обусловлена тем, что существующие в настоящее время популярные сервисы антиплагиата очень плохо  распознают сгенерированный русскоязычный текст.

**Цели проекта**
1.	Выявление отличий  в статистических  характеристиках русскоязычного текста написанного человеком,  и текста сгенерированного большими языковыми моделями.

2.	Создание системы распознавания русскоязычного текста сгенерированного большими языковыми моделями.

**В качестве данных взяты**  

50 дипломных работ студентов специальности 09.02.07 "Информационные системы" объемом 40 – 50стр. выполненных в 2019 - 2020 гг. , и 50 дипломных работ студентов той же специальности объемом 40 – 50стр,  выполненных в 2024г.

##В основном датасете присутствуют следующие столбцы

**file_number**	- номер файла (1 - 100) Каждый файл представляет собой документ (дипломную работу студента в формате txt, очищенную от фрагментов программного кода и с удаленным экономическим разделом.)

**paragraph_number**	- номер абзаца в документе

**content**	- содержимое абзаца

**label** - метка класса

(0 - работа за 2019-2020гг.

1 - работа за 2024 г.)

## Работа с задачей естественного языка
--------

Будем работать следующим образом:

1. Классификация текста с помощью LSTM
2. Классификация текста с помощью предобученной на данных русскоязычных твитов модели BERT

In [ ]:
!pip -q install pymorphy2
!pip -q install sentencepiece
!pip -q install transformers

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 60.6 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade gdown

In [ ]:
!gdown 1kD5QBEvBQCGaAzsYskIBCDo9sOeCwqKP  #bert_dataset.py
!gdown 1N_eGBLKUuGzlL5dLzQvC4UfcZdZXf6eS  #bert_classifier.py

Downloading...
From (original): https://drive.google.com/uc?id=1kD5QBEvBQCGaAzsYskIBCDo9sOeCwqKP
From (redirected): https://drive.google.com/uc?id=1kD5QBEvBQCGaAzsYskIBCDo9sOeCwqKP&confirm=t&uuid=55181428-99fe-4e31-92ab-950c212e92e7
To: /content/bert_dataset.py
100% 891/891 [00:00<00:00, 4.12MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1N_eGBLKUuGzlL5dLzQvC4UfcZdZXf6eS
From (redirected): https://drive.google.com/uc?id=1N_eGBLKUuGzlL5dLzQvC4UfcZdZXf6eS&confirm=t&uuid=d99d581f-3abc-4a84-b3e2-7c586b2c8eed
To: /content/bert_classifier.py
100% 5.28k/5.28k [00:00<00:00, 18.3MB/s]


In [ ]:
# основные по работе с данными (таблицы, математика)
import pandas as pd
import numpy as np

In [ ]:
# модели
import sklearn

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
!pip -q install keras --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.8 MB/s eta 0:00:00


In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
#from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Embedding, Dropout, GlobalAveragePooling1D, Flatten, SpatialDropout1D, Bidirectional

Датасет с очищенным текстом

In [ ]:
!gdown 11KFmtEdK_h1S5RyfRfHSp4fJBDEyo0NI

Downloading...
From: https://drive.google.com/uc?id=11KFmtEdK_h1S5RyfRfHSp4fJBDEyo0NI
To: /content/clean_df.csv
100% 6.87M/6.87M [00:00<00:00, 28.5MB/s]


Датасет с исходным текстом

In [ ]:
!gdown 1-gq9OHKFFzpWKh9enhdPw6Cjcef7vMFr

Downloading...
From: https://drive.google.com/uc?id=1-gq9OHKFFzpWKh9enhdPw6Cjcef7vMFr
To: /kaggle/working/ish_df.csv
100%|██████████████████████████████████████| 7.91M/7.91M [00:00<00:00, 31.5MB/s]


In [ ]:
df = pd.read_csv('clean_df.csv')

In [ ]:
df

,text,label
0,введение цель выпускной квалификационный работ...,0
1,работа представить ниже стоить считать предста...,0
2,компьютерный игра правило игровой ситуация вос...,0
3,сразу стоить отметить сегодняшний день существ...,0
4,вовторое разработчик прекращать свой экспериме...,0
...,...,...
5920,воздействие внешний условие превращаться орган...,1
5921,современный настольный пк такой компонент прак...,1
5922,стоимость такой услуга большой это безусловно ...,1
5923,использовать пример разработка аналогичный сис...,1


##  Классификация текста с помощью LSTM

In [ ]:
df['text'] = df['text'].astype(str)

Делим на train и test

In [ ]:
TEST_SIZE = 0.25
RANDOM_STATE = 1234

x_train, x_test, y_train, y_test = train_test_split(df['text'], df['label'],
                                                    test_size=TEST_SIZE, random_state=RANDOM_STATE)

In [ ]:
# Параметры препроцессинга
max_len = 100 # размер вектора слов, 512
trunc_type = 'post' # обрезка до вектора
padding_type = 'post' # добавление до вектора
oov_tok = '<OOV>' # нет в словаре
vocab_size = 1500 # размер словаря (зависит от входа)

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size,
                      char_level = False,
                      oov_token = oov_tok)
tokenizer.fit_on_texts(x_train)

In [ ]:
# Получить индекс слова
word_index = tokenizer.word_index
total_words = len(word_index)
total_words

15681

In [ ]:
training_sequences = tokenizer.texts_to_sequences(x_train) # векторизация
training_padded = pad_sequences(training_sequences,
                                maxlen = max_len,
                                padding = padding_type,
                                truncating = trunc_type)
testing_sequences = tokenizer.texts_to_sequences(x_test)
testing_padded = pad_sequences(testing_sequences,
                               maxlen = max_len,
                               padding = padding_type,
                               truncating = trunc_type)

print('Shape of training tensor: ', training_padded.shape)
print('Shape of testing tensor: ', testing_padded.shape)

Shape of training tensor:  (4443, 100)
Shape of testing tensor:  (1482, 100)


In [ ]:
# Параметры
embedding_dim = 32 # Размер вектора эмбедднгов 16, 32
n_dense = 24 # Полносвязный слой
n_lstm = 64 # слой LTSM
drop_lstm = 0.1 # слой Dropout

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(SpatialDropout1D(drop_lstm))
model.add(LSTM(n_lstm, return_sequences=False))
model.add(Dropout(drop_lstm))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(loss = 'binary_crossentropy',
               optimizer = 'adam',
               metrics = ['accuracy'])

In [ ]:
num_epochs = 10
early_stop = EarlyStopping(monitor='val_loss', patience=3) # остановиться, когда метрика перестает улучшаться, помогает выйти из цикла с эпохами
history = model.fit(training_padded,
                    y_train,
                    epochs=num_epochs,
                    validation_data=(testing_padded, y_test),
                    callbacks =[early_stop],
                    verbose=2)

Epoch 1/10
139/139 - 10s - 69ms/step - accuracy: 0.5291 - loss: 0.6922 - val_accuracy: 0.5135 - val_loss: 0.6927
Epoch 2/10
139/139 - 1s - 7ms/step - accuracy: 0.5472 - loss: 0.6866 - val_accuracy: 0.5155 - val_loss: 0.6870
Epoch 3/10
139/139 - 1s - 9ms/step - accuracy: 0.5762 - loss: 0.6605 - val_accuracy: 0.7395 - val_loss: 0.6016
Epoch 4/10
139/139 - 1s - 7ms/step - accuracy: 0.6424 - loss: 0.6430 - val_accuracy: 0.6754 - val_loss: 0.6244
Epoch 5/10
139/139 - 1s - 7ms/step - accuracy: 0.6905 - loss: 0.6160 - val_accuracy: 0.7206 - val_loss: 0.6002
Epoch 6/10
139/139 - 1s - 7ms/step - accuracy: 0.6406 - loss: 0.6414 - val_accuracy: 0.6161 - val_loss: 0.6502
Epoch 7/10
139/139 - 1s - 9ms/step - accuracy: 0.5935 - loss: 0.6576 - val_accuracy: 0.6154 - val_loss: 0.6532
Epoch 8/10
139/139 - 2s - 12ms/step - accuracy: 0.5935 - loss: 0.6561 - val_accuracy: 0.6188 - val_loss: 0.6514


In [ ]:
# сохранить модель в файл
model.save('model_lstm.keras') # более новая версия файла
model.save('model_lstm.h5') # чаще h5, более старая версия

In [ ]:
import pickle
with open('tokenizer.pickle', 'wb') as handle:
  # выгрузить файл с дампом модели расширения pickle
  pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Классификация текста с помощью предобученной на данных русскоязычных твитов модели BERT
----

Для эксперимента была использована предобученная модель Bert на данных сообщений из русскоязычного твитера

In [ ]:
TEST_SIZE_BERT = 0.2 # 20% тестовой
RANDOM_STATE_BERT = 1 # воспроизводимость модели

train, test_data = train_test_split(df, test_size=TEST_SIZE_BERT, random_state=RANDOM_STATE_BERT)
train_data, valid_data = train_test_split(train, test_size=TEST_SIZE_BERT, random_state=RANDOM_STATE_BERT)

In [ ]:
from bert_dataset import CustomDataset
from bert_classifier import BertClassifier

In [ ]:
EPOCH_COUNT_BERT = 6
CLASS_LABELS = 2

# https://huggingface.co/cointegrated/rubert-tiny ruBERT
classifier = BertClassifier(
        model_path='cointegrated/rubert-tiny',
        tokenizer_path='cointegrated/rubert-tiny',
        n_classes=CLASS_LABELS,
        epochs=EPOCH_COUNT_BERT,
        model_save_path='bert.pt'
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/468k [00:00<?, ?B/s]

In [ ]:
classifier.preparation(
        X_train=list(train_data['text']),
        y_train=list(train_data['label']),
        X_valid=list(valid_data['text']),
        y_valid=list(valid_data['label'])
    )

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
classifier.train()

Epoch 1/6
Train loss 0.7179656924096817 accuracy 0.6890822784810126
Val loss 0.8464543829723123 accuracy 0.729957805907173
----------
Epoch 2/6
Train loss 0.7358100618891401 accuracy 0.7808544303797468
Val loss 0.9742800498802219 accuracy 0.7573839662447257
----------
Epoch 3/6
Train loss 0.6481320179708696 accuracy 0.8349156118143459
Val loss 0.9767410650238776 accuracy 0.7616033755274261
----------
Epoch 4/6
Train loss 0.5662819562397086 accuracy 0.8636603375527425
Val loss 1.0234615773566504 accuracy 0.7995780590717299
----------
Epoch 5/6
Train loss 0.49054706496240463 accuracy 0.8842299578059071
Val loss 1.1063420572289893 accuracy 0.790084388185654
----------
Epoch 6/6
Train loss 0.43441119646860576 accuracy 0.9026898734177214
Val loss 1.1113061679560075 accuracy 0.7921940928270041
----------


/content/bert_classifier.py:112: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model = torch.load(self.model_save_path)


In [ ]:
texts = list(test_data['text'])
labels = list(test_data['label'])

predictions = [classifier.predict(t) for t in texts]

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1score = precision_recall_fscore_support(labels,
                                                             predictions,
                                                             average='macro')[:3]

print(f'precision: {precision:.3f}, recall: {recall:.3f}, f1score: {f1score:.3f}')

precision: 0.793, recall: 0.793, f1score: 0.792


In [ ]:
# сохранить модель в файл
model.save('model_bert.keras') # более новая версия файла

## Выводы
----
**Модель LTSM** требует большего набора данных и продолжения тренировок. А еще можно с архитектурой поэксперементировать, параметры настроить... LSTM  вообще "из коробки" плохо работает. Ее настраивать надо. Пока результат неважный.


**Модель BERT** вроде бы показывает сравнимый результат с классическим машинным обучением.
Но меня сильно смущает то, что **Train loss** уменьшается последовательно от эпохи к эпохе, а **Val loss** так же последовательно растет.
Похоже на ситуацию с переобучением.

Хотелось еще на неочищенном тексте модели запустить, но ГПУ стало недоступно. Потом сделаю. хотя навряд ли это улучшит качество.
